In [ ]:
import pandas as pd
import re
import sqlite3 
from IPython.display import Markdown, display
from ipywidgets import  widgets,Layout
from PIL import Image

def displayRecord(change):    
    value=change['new']
    df=searchGua(change)    
    text_box.value=str(len)
    # print( f'当前数据编号: {value}')
    content=df.loc[value,'guaContent']
    content=content.replace("\n","\n\n")
    text_box.value=content
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    images=re.findall(re1,content)
    img_dropdown.options=images
    img_dropdown.index=1
    displayImage(change)

def displayImages(change):
    content=""
    for item in img_dropdown.options:
        content=content+"![]("+item+")"   
    # content=img_dropdown.value
    # content="![]("+content+")"    
    with output2:
        output2.clear_output()
        display(Markdown(content))
        # display(Image.open(content))

def searchGua(change):
    gua_name_str=gua_name_text.value
    database="Guas.db"
    connection=sqlite3.connect(database)
    # gua_name_str="泰静卦"
    query=f'select rowid, guaDate, guaName,stockName,substr(guaSubject,1,20),guaContent from StockGuas  where guaName like "{gua_name_str}"   '
    df=pd.read_sql_query(query,connection)
    df_display=df[['rowid', 'guaDate','guaName', 'stockName','substr(guaSubject,1,20)']]
    output1.clear_output( )
    output2.clear_output()
    text_box.value=""
    length=df.shape[0]
    slider.max=length
    with output1:
        print(df_display)
    connection.close()
    return df

slider = widgets.IntSlider(value=0, min=0, max=20, description='查询编号')
gua_name_text = widgets.Text(description='卦名:', layout=Layout(width='150px', height='20px'))
monthday_text = widgets.Text(description='日月:', layout=Layout(width='150px', height='20px'))
xunkong_text = widgets.Text(description='旬空:', layout=Layout(width='150px', height='20px'))
button_search = widgets.Button(description='搜索', layout=Layout(width='100px', height='20px'))
output1=widgets.Output(layout=Layout(width='80%', height='200px'))
output2=widgets.Output(layout=Layout(width='80%', height='200px'))

img_dropdown = widgets.Dropdown(discription='选择图片',layout=Layout(width='300px', height='20px'))
text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='50%', height='400px'))

search_bar=widgets.HBox([button_search,gua_name_text,monthday_text,xunkong_text])
search_box=widgets.VBox([search_bar,output1])
image_box=widgets.VBox([img_dropdown,output2])
detail_box=widgets.HBox([text_box,image_box])
main_layout=widgets.VBox([search_box,slider,detail_box])
button_search.on_click(searchGua)
slider.observe(displayRecord,names='value')
img_dropdown.observe(displayImages,names='value')
# widgets.interact(displayRecord, value=slider)
# button_disp.on_click(displayRecord)
display(main_layout)
                  

In [ ]:
# 最简单搜索程序，按卦名+干支+旬空，搜索相关记录到df
#下一个程序则显示各记录细节
import sqlite3 
import pandas as pd

def loadinfo(gua_name_str,gangzhi_str,xunkong_str):
    database="Guas.db"
    connection=sqlite3.connect(database)
    query=f'select rowid, guaDate, guaName, stockName, substr(guaSubject,1,20) as subject, guaContent from StockGuas  where guaName like "{gua_name_str}"   '
    orderby= '  order by cast( substr(guaDate,6,2) as integer),guaDate '
    sub2=""
    sub3=""
    if gangzhi_str!="":
        sub2=f' and gangZhi like "%{gangzhi_str}%"  '
    if xunkong_str!="":
        sub3=f' and xunKong like "%{xunkong_str}%"  '
    query=query+sub2+sub3+orderby
    df=pd.read_sql_query(query,connection)
    return df
# df_disp=df[['rowid', 'guaDate','guaName','stockName','subject']]
# df_disp=df_disp.rename(columns = {'rowid':'记录号' ,'guaDate':'日期','guaName':'卦名','stockName':'股票','subject':'预测主题'})
gua_name_str=input("输入卦名:")
gangzhi_str=input("输入日月干支:")
xunkong_str=input("输入旬空:")
df=loadinfo(gua_name_str,gangzhi_str,xunkong_str)
df.head(10)


In [ ]:
import ipywidgets as widgets
from IPython.display import HTML,display,Markdown
import sixyao
import sqlite3
import akshare_plotly as akPlot

def draw_daily_35(change):
    stockCode=stock_name_text.value
    dateOrigin=gua_date_text.value
    days=35
    imgfile=akPlot.drawDailyLine(stockCode,dateOrigin,time_period=days)
    pics_content="![]("+imgfile+")" 
    output.clear_output()
    with output:
        display(Markdown(pics_content)) 
    # temp=stockCode+" ---   OK  ---  "  +dateOrigin +"    "+imgfile
    # with output:
    #     print(temp)

def draw_daily_15(change):
    stockCode=stock_name_text.value
    dateOrigin=gua_date_text.value
    days=15
    imgfile=akPlot.drawDailyLine(stockCode,dateOrigin,time_period=days)
    pics_content="![]("+imgfile+")" 
    output.clear_output()
    with output:
        display(Markdown(pics_content)) 

def draw_daily_365(change):
    stockCode=stock_name_text.value
    dateOrigin=gua_date_text.value
    imgfile=akPlot.drawMonthLine(stockCode,dateOrigin)
    pics_content="![]("+imgfile+")" 
    output.clear_output()
    with output:
        display(Markdown(pics_content)) 
        
def paipan(change):
    day1=gua_date_text.value
    namestr=gua_name_text.value
    cont=sixyao.paipan(day1,namestr)
    temp=text_box.value+cont
    text_box.value=temp

def save_text(change):
    new_content = text_box.value
    new_stockCode= stock_name_text.value
    new_guaName=  gua_name_text.value
    new_guaDate= gua_date_text.value
    new_subject= subject_text.value
    id=slider1.value
    rowid=df.loc[id,'rowid']
    connection = sqlite3.connect('Guas.db')
    cursor = connection.cursor()
    query='UPDATE StockGuas  SET guaDate="{}", stockName = "{}", guaName="{}", guaContent ="{}",guaSubject="{}" WHERE rowid = {}  '.format( new_guaDate, new_stockCode, new_guaName, new_content, new_subject, rowid)
    cursor.execute(query)
    connection.commit()
    cursor.close()
    connection.close() 
        
def record_show(change):
    value=change['new']
    # print( f'当前数据编号: {value}')
    gua_name_text.value=df.loc[value,'guaName']
    gua_date_text.value=df.loc[value,'guaDate']
    stock_name_text.value=df.loc[value,'stockName']
    subject_text.value=df.loc[value,'subject']
    text_box.value=df.loc[value,'guaContent']
    # content=content.replace("\n","\n\n")
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    content=text_box.value
    image_show(change)
        
def image_show(change):
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    content=text_box.value
    images=re.findall(re1,content)
    pics_content=""
    for item in images:
        item1="https://64gua.github.io/"+item
        pics_content=pics_content+"![]("+item+")" 
    with output:
        output.clear_output()
        display(Markdown(pics_content))
        # for item in images:
        #     print(item)  
length=df.shape[0]
print("df length=", length)
slider1 = widgets.IntSlider(value=0, min=0, max=length-1, description='卦例',layout=Layout(width='80%', height='20px'))
gua_name_text = widgets.Text(description='卦名:', layout=Layout(width='150px', height='20px'))
gua_date_text = widgets.Text(description='日期:', layout=Layout(width='150px', height='20px'))
stock_name_text = widgets.Text(description='股票:', layout=Layout(width='150px', height='20px'))
subject_text = widgets.Text(description='钥语:', layout=Layout(width='200px', height='20px'))
button_sixyao = widgets.Button(description='生成六爻', layout=Layout(width='100px', height='20px'))
button_draw15 = widgets.Button(description='绘图15天', layout=Layout(width='100px', height='20px'))
button_draw35 = widgets.Button(description='绘图35天', layout=Layout(width='100px', height='20px'))
button_draw365 = widgets.Button(description='绘图一年', layout=Layout(width='100px', height='20px'))
button_save_img = widgets.Button(description='图片存盘', layout=Layout(width='100px', height='20px'))
button_save_text = widgets.Button(description='保存修改', layout=Layout(width='100px', height='20px'))
text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='100%', height='200px'))
# text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='50%', height='400px'))
output=widgets.Output(layout=Layout(width='80%', height='500px'))
layout1=widgets.HBox([gua_name_text,gua_date_text])
layout2=widgets.HBox([stock_name_text,subject_text])
layout3=widgets.HBox([button_sixyao,button_save_text])
layout4=widgets.HBox([button_draw15,button_draw35,button_draw365,button_save_img ])
slider1.observe(record_show, names='value')
# slider2.observe(image_show,names='value')
button_sixyao.on_click(paipan)
button_save_text.on_click(save_text)
button_draw15.on_click(draw_daily_15)
button_draw35.on_click(draw_daily_35)
button_draw365.on_click(draw_daily_365)
# widgets.interact(format_slider, value=slider1)
# widgets.interact(image_show, value=slider2)
display(slider1, layout1 ,layout2,layout3,text_box,layout4,output)

# df